### Task 0 - Data ingestion


In [66]:
import duckdb
import pandas as pd

# "r" - read mode for the file we open
with open("sql/load_sakila.sql", "r") as sql_file:
    loading_script = sql_file.read()

print(loading_script)



INSTALL sqlite;

LOAD sqlite;

CALL sqlite_attach ('data/sqlite-sakila.db');


### Function for SQL queries

In [67]:
def query_sakila(query: str, duckdb_path="data/sakila.duckdb"):
    with duckdb.connect(duckdb_path) as conn:
        df = conn.sql(query=query).df()

    return df

In [68]:
import duckdb
from pathlib import Path

duckdb_path = "data/sakila.duckdb"
Path(duckdb_path).unlink(missing_ok=True)

with duckdb.connect(duckdb_path) as conn, open("sql/load_sakila.sql") as ingest_script:
    conn.sql(ingest_script.read())

    description = conn.sql("DESC;").df()
    films = conn.sql("FROM film;").df()

films.head(3)

description

,database,schema,name,column_names,column_types,temporary
0,sakila,main,actor,"[actor_id, first_name, last_name, last_update]","[DOUBLE, VARCHAR, VARCHAR, TIMESTAMP]",False
1,sakila,main,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, VA...",False
2,sakila,main,category,"[category_id, name, last_update]","[BIGINT, VARCHAR, TIMESTAMP]",False
3,sakila,main,city,"[city_id, city, country_id, last_update]","[BIGINT, VARCHAR, BIGINT, TIMESTAMP]",False
4,sakila,main,country,"[country_id, country, last_update]","[BIGINT, VARCHAR, TIMESTAMP]",False
5,sakila,main,customer,"[customer_id, store_id, first_name, last_name,...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, BI...",False
6,sakila,main,customer_list,"[ID, name, address, zip_code, phone, city, cou...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, VARCHAR, V...",False
7,sakila,main,film,"[film_id, title, description, release_year, la...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, BI...",False
8,sakila,main,film_actor,"[actor_id, film_id, last_update]","[BIGINT, BIGINT, TIMESTAMP]",False
9,sakila,main,film_category,"[film_id, category_id, last_update]","[BIGINT, BIGINT, TIMESTAMP]",False


### Task 1 - EDA in python
The Manager would like you to explore the data and analyze the questions below. He wants you to make it in
jupyter notebook so that the team can directly see the results of your EDA. Combine duckdb and pandas to
do the EDA.
It's important that you show the answers, i.e. run your cells with outputs directly in the notebook.

#### a) Which movies are longer than 3 hours (180 minutes), show the title and its length?

In [69]:
query_sakila ("""
SELECT title, length
FROM film
WHERE length > 180
ORDER BY length DESC;
""")

,title,length
0,WORST BANGER,185
1,CHICAGO NORTH,185
2,CONTROL ANTHEM,185
3,DARN FORRESTER,185
4,SWEET BROTHERHOOD,185
5,GANGS PRIDE,185
6,HOME PITY,185
7,SOLDIERS EVOLUTION,185
8,POND SEATTLE,185
9,MUSCLE BRIGHT,185


#### b) Which movies have the word "love" in its title? Show the following columns
- title
- rating
- length
- description

In [70]:
query_sakila("""
SELECT 
    title,
    rating,
    length,
    description
FROM film
WHERE LOWER(title) LIKE '%love%';
""")

,title,rating,length,description
0,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
1,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
2,IDENTITY LOVER,PG-13,119,A Boring Tale of a Composer And a Mad Cow who ...
3,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
4,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...
5,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explore...
6,LOVELY JINGLE,PG,65,A Fanciful Yarn of a Crocodile And a Forensic ...
7,LOVER TRUMAN,G,75,A Emotional Yarn of a Robot And a Boy who must...
8,LOVERBOY ATTACKS,PG-13,162,A Boring Story of a Car And a Butler who must ...
9,STRANGELOVE DESIRE,NC-17,103,A Awe-Inspiring Panorama of a Lumberjack And a...


#### c) Calculate descriptive statistics on the length column, The Manager wants, shortest, average, median and longest movie length

In [71]:
query_sakila("""
SELECT
    MIN(length) AS shortest,
    AVG(length) AS average,
    MEDIAN(length) AS median,
    MAX(length) AS longest
FROM film;
""")

,shortest,average,median,longest
0,46,115.272,114.0,185


#### d) The rental rate is the cost to rent a movie and the rental duration is the number of days a customer can keep the movie. The Manager wants to know the 10 most expensive movies to rent per day

In [72]:
query_sakila("""
SELECT
    title,
    rental_rate,
    rental_duration,
    rental_rate / rental_duration AS price_per_day
FROM film
ORDER BY price_per_day DESC
LIMIT 10;
""")


,title,rental_rate,rental_duration,price_per_day
0,BACKLASH UNDEFEATED,4.99,3,1.663333
1,BILKO ANONYMOUS,4.99,3,1.663333
2,BEAST HUNCHBACK,4.99,3,1.663333
3,AUTUMN CROW,4.99,3,1.663333
4,ACE GOLDFINGER,4.99,3,1.663333
5,CARIBBEAN LIBERTY,4.99,3,1.663333
6,BEHAVIOR RUNAWAY,4.99,3,1.663333
7,CASPER DRAGONFLY,4.99,3,1.663333
8,AMERICAN CIRCUS,4.99,3,1.663333
9,CASUALTIES ENCINO,4.99,3,1.663333


### e) Which actors have played in most movies? Show the top 10 actors with the number of movies they have played in.

In [73]:
query_sakila("""
SELECT
    a.actor_id,
    a.first_name,
    a.last_name,
    COUNT(fa.film_id) AS num_movies
FROM actor AS a
JOIN film_actor AS fa ON a.actor_id = fa.actor_id
GROUP BY a.actor_id, a.first_name, a.last_name
ORDER BY num_movies DESC
LIMIT 15;
""")

,actor_id,first_name,last_name,num_movies
0,107.0,GINA,DEGENERES,42
1,102.0,WALTER,TORN,41
2,198.0,MARY,KEITEL,40
3,181.0,MATTHEW,CARREY,39
4,23.0,SANDRA,KILMER,37
5,81.0,SCARLETT,DAMON,36
6,60.0,HENRY,BERRY,35
7,37.0,VAL,BOLGER,35
8,158.0,VIVIEN,BASINGER,35
9,13.0,UMA,WOOD,35


#### f) Now it's time for you to choose your own question to explore the sakila database! Write down 3-5 questions you want to answer and then answer them using pandas and duckdb

#### Question 1
- Which customers have rented the most movies?
- Join customer -> rental, count number of rentals per customer,
- and display the top 10. This identifies our most active customers.

#### Which customers have rented the most movies?

In [74]:
query_sakila("""
SELECT
    c.customer_id,
    c.first_name,
    c.last_name,
    COUNT(r.rental_id) AS num_rentals
FROM customer c
JOIN rental r
    ON c.customer_id = r.customer_id
GROUP BY 
    c.customer_id, 
    c.first_name, 
    c.last_name
ORDER BY num_rentals DESC
LIMIT 10;
""")

,customer_id,first_name,last_name,num_rentals
0,148,ELEANOR,HUNT,46
1,526,KARL,SEAL,45
2,144,CLARA,SHAW,42
3,236,MARCIA,DEAN,42
4,75,TAMMY,SANDERS,41
5,469,WESLEY,BULL,40
6,197,SUE,PETERS,40
7,137,RHONDA,KENNEDY,39
8,178,MARION,SNYDER,39
9,468,TIM,CARY,39


#### Question 2 (Follow-up question)

**Which customers have spent the most money?**

This follow-up question builds on Question 1. After identifying which customers rent the most movies, we now explore which customers generate the highest revenue.

- Join customer -> rental -> payment  
- Sum payment amounts per customer  
- Display the top 10 highest-spending customers  
- Helps identify our most valuable and profitable customers


In [75]:
query_sakila("""
SELECT
    c.customer_id,
    c.first_name,
    c.last_name,
    SUM(p.amount) AS total_spent
FROM customer c
JOIN rental r ON c.customer_id = r.customer_id
JOIN payment p ON r.rental_id = p.rental_id
GROUP BY 
    c.customer_id, 
    c.first_name, 
    c.last_name
ORDER BY total_spent DESC
LIMIT 10;
""")


,customer_id,first_name,last_name,total_spent
0,526,KARL,SEAL,221.55
1,148,ELEANOR,HUNT,216.54
2,144,CLARA,SHAW,195.58
3,178,MARION,SNYDER,194.61
4,137,RHONDA,KENNEDY,194.61
5,459,TOMMY,COLLAZO,186.62
6,469,WESLEY,BULL,177.60
7,468,TIM,CARY,175.61
8,236,MARCIA,DEAN,175.58
9,181,ANA,BRADLEY,174.66


#### Question 3

**Which cities generate the most rentals?**

This question identifies which city rents the most films.

- Join city → address → customer → rental  
- Count how many rentals each city generates  
- Display the top 10 cities with the highest rental activity  

In [76]:
query_sakila("""
SELECT
    ci.city,
    COUNT(r.rental_id) AS num_rentals
FROM city ci
JOIN address a ON ci.city_id = a.city_id
JOIN customer c ON a.address_id = c.address_id
JOIN rental r ON c.customer_id = r.customer_id
GROUP BY ci.city
ORDER BY num_rentals DESC
LIMIT 10;
""")


,city,num_rentals
0,Aurora,50
1,London,48
2,Saint-Denis,46
3,Cape Coral,45
4,Tanza,42
5,Molodetno,42
6,Changhwa,41
7,Ourense (Orense),40
8,Changzhou,40
9,Santa Brbara dOeste,39
